In [115]:
import pandas as pd
import numpy as np
import time
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

In [116]:
# preprocessing doc

def preprocessing():
    # read data
    # TODO: in the script that generates non-phrases, remove ',' in numbers
    file = pd.read_csv('NSF_awardtopics.txt', error_bad_lines=False)
    file.columns = ['NSF Award Title Non-phrases']
    documents = file['NSF Award Title Non-phrases']
    
    word2id = {}
    id2word = {}
    docs = []
    currentDocument = []
    currentWordId = 0
    
    for document in documents:
        segList = gensim.utils.simple_preprocess(document)
        for word in segList: 
            if len(word) >= 3 and word not in gensim.parsing.preprocessing.STOPWORDS:
                if word in word2id:
                    currentDocument.append(word2id[word])
                else:
                    currentDocument.append(currentWordId)
                    word2id[word] = currentWordId
                    id2word[currentWordId] = word
                    currentWordId += 1
        docs.append(currentDocument)
        currentDocument = []
    return docs, word2id, id2word


In [117]:
# Go through each document & randomly assign each word in the document to a topic z
# gives you the topic representation of all documents & word distributions of all topics, though not good ones

def randomInitialize():
    # d: document index, doc: document content
    for d, doc in enumerate(docs): 
        zCurrentDoc = []   # topics for each word in the current document
        for w in doc:  # w: each word in the document
            pz = np.divide(np.multiply(ndz[d, :], nzw[:, w]), nz)

            # draw samples from a multinomial distribution; 
            # (n, pvals, size=None): n is num of experiments, pvals is probabilities of each of the p different outcomes
            # randomly sample a topic for word w. z is the topic index 
            z = np.random.multinomial(1, pz / pz.sum()).argmax()
            
            zCurrentDoc.append(z)  # add this word's topic to zCurrentDoc
            
            # update variables
            ndz[d, z] += 1
            nzw[z, w] += 1
            nz[z] += 1
            
        Z.append(zCurrentDoc)

In [118]:
# Improve by using gibbs sampling:
# for each doc d, 
#    for each word w, 
#         for each topic z, compute two things: 1. p(topic z| doc d) 2. p(word w| topic z)
#         we reassign w a new topic z with probability p(topic z| doc d) * p(word w| topic z) (probability that topic z generated w)

def gibbsSampling():
    # regenerate topics for each word in each document
    for d, doc in enumerate(docs):
        for index, w in enumerate(doc):
            z = Z[d][index]
            # decrement the topic cnt for this word in this doc
            ndz[d, z] -= 1
            nzw[z, w] -= 1
            nz[z] -= 1
            
            # recalculate the probability of each word w belonging to each topic z
            pz = np.divide(np.multiply(ndz[d, :], nzw[:, w]), nz)
            
            if (w==104 and flag==True):
                print(pz)
                
            # resample from the updated distribution
            z = np.random.multinomial(1, pz / pz.sum()).argmax()
            
            Z[d][index] = z 
            
            # update variables
            ndz[d, z] += 1
            nzw[z, w] += 1
            nz[z] += 1
            

In [119]:
# log response
log = False
flag = False

# define hyperparameters
alpha = 5
beta = 0.1

# define iteration number
iterationNum = 10

# define how many topics we want to generate
K = 35  # number of topics

# preprocess
docs, word2id, id2word = preprocessing()
print(word2id['robotics'])
# print(docs)

# variables
Z = []  # list of lists. Z[i,j] means the topic index of [ith document, jth word]
N = len(docs) # total number of documents
M = len(word2id) # length of word list

ndz = np.zeros([N, K]) + alpha  # ndz[i,z] means in the ith document, number of words generated by topic z
nzw = np.zeros([K, M]) + beta   # nzw[z,w] means the number of word w generated by topic z
nz = np.zeros([K]) + M * beta   # nz[z] means the number of each words generated by topic z

if log:
    print("*****Main******")
    print("ndz:")
    print(ndz)
    print("==================================")
    print("nzw:")
    print(nzw)
    print("==================================")
    print("nz:")
    print(nz)
    print("==================================")

# initialize
randomInitialize()
if log:
    print("*****Initialize******")
    print("ndz:")
    print(ndz)
    print("==================================")
    print("nzw:")
    print(nzw)
    print("==================================")
    print("nz:")
    print(nz)
    print("==================================")

# realizing the pgm by gibbs sampling
for i in range(0, iterationNum):
    if i==iterationNum-1:
        flag = True
    gibbsSampling()
    
    print("Iteration {} completed.".format(i+1))
    if log: 
        print("*****Sampling******")
        print("ndz:")
        print(ndz)
        print("==================================")
        print("nzw:")
        print(nzw)
        print("==================================")
        print("nz:")
        print(nz)
        print("==================================")
    
# show results
topicwords = []
maxTopicWordsNum = 5 # let's say we want to show max top 10 words that contribute to each topic

for z in range(0, K): # for each topic
    ids = nzw[z, :].argsort()  # number of each word generated by topic z, ranked
    topicword = []
    for j in ids:
        topicword.insert(0, id2word[j])  # in decrementing order
    topicwords.append(topicword[0 : min(maxTopicWordsNum, len(topicword))])
    
print(topicwords)

b'Skipping line 946: expected 1 fields, saw 2\nSkipping line 1957: expected 1 fields, saw 2\nSkipping line 2826: expected 1 fields, saw 2\nSkipping line 4376: expected 1 fields, saw 2\nSkipping line 5810: expected 1 fields, saw 3\nSkipping line 5811: expected 1 fields, saw 3\n'


104
Iteration 1 completed.
Iteration 2 completed.
Iteration 3 completed.
Iteration 4 completed.
Iteration 5 completed.
Iteration 6 completed.
Iteration 7 completed.
Iteration 8 completed.
Iteration 9 completed.
[7.57108208e-03 2.32351905e-05 2.71624196e-05 3.43998563e-05
 1.71438956e-05 2.76287797e-05 3.03964741e-01 2.73620270e-05
 3.10319280e-05 1.91630353e-05 1.17172943e-03 3.11079485e-05
 3.11685478e-05 8.67887758e-05 2.84519305e-05 2.73952231e-05
 2.37098858e-05 1.99029532e-05 1.94190594e-05 6.48041200e-05
 2.40481926e-05 1.84870905e-05 2.01199957e-05 2.41197303e-05
 1.98208984e-05 2.75672900e-05 2.78596986e-05 6.86641953e-04
 2.36780729e-05 2.78098916e-05 2.82145434e-05 3.11782656e-05
 3.15081837e-03 2.72809824e-05 2.15797064e-05]
[7.57137971e-03 1.93626587e-05 1.94062465e-05 2.29402521e-05
 1.71444834e-05 1.97356216e-05 3.95168775e-01 2.34494995e-05
 5.04210155e-05 2.68261931e-05 1.64061029e-03 3.49855587e-05
 3.50618842e-05 5.91694969e-05 3.25085640e-05 1.95710802e-05
 2.7667000

[1.17182429e-02 3.48473878e-05 2.33010614e-05 2.29341141e-05
 2.05931514e-05 1.97558961e-05 2.42365136e-01 3.12794467e-05
 3.10295207e-05 2.68632545e-05 2.17934305e-03 3.49787601e-05
 1.95046597e-05 2.35914898e-05 3.65959281e-05 2.73716563e-05
 2.37493024e-05 2.77811953e-05 2.32947288e-05 1.90840423e-05
 3.60708431e-05 2.58847979e-05 2.01127116e-05 2.00884696e-05
 3.96182387e-05 2.06776051e-05 3.58938976e-05 5.15318455e-04
 2.36510058e-05 2.78552640e-05 2.01499966e-05 2.72554891e-05
 2.62441952e-03 2.33382214e-05 2.52271343e-05]
[7.81124016e-03 2.32324914e-05 2.71856273e-05 3.05776500e-05
 1.71615485e-05 3.95133535e-05 3.02922849e-01 1.95496542e-05
 2.32721405e-05 1.91880389e-05 1.36203715e-03 2.33137368e-05
 3.12111080e-05 3.14540829e-05 4.06671032e-05 2.73727266e-05
 2.77086162e-05 1.98413486e-05 2.32956332e-05 3.43538986e-05
 4.80963852e-05 2.95826261e-05 2.41362249e-05 2.00876626e-05
 2.37756529e-05 2.06790304e-05 3.98821085e-05 4.29484741e-04
 2.36491413e-05 2.38759406e-05 2.01499

[7.59047683e-03 2.32414907e-05 2.33700373e-05 3.43945978e-05
 2.40393696e-05 1.97449739e-05 2.12351899e-01 2.34742702e-05
 3.48852083e-05 2.53262676e-04 2.72941502e-03 2.33509373e-05
 3.51742493e-05 1.96820173e-05 3.65231577e-05 1.94894543e-05
 1.97957867e-05 3.96119612e-05 3.49475205e-05 2.29262272e-05
 3.19796609e-05 2.59250618e-05 2.41546866e-05 4.00706847e-05
 2.77988475e-05 2.06875864e-05 3.18955103e-05 4.29590171e-04
 3.54067249e-05 2.38152886e-05 2.81725286e-05 3.50782830e-05
 2.61969612e-03 2.71782388e-05 1.80310784e-05]
[1.01202369e-02 1.93694095e-05 7.79152983e-05 2.67452212e-05
 2.40410209e-05 1.97465335e-05 3.03348635e-01 1.95611266e-05
 5.03819338e-05 3.79879436e-04 1.91088435e-03 3.89136856e-05
 3.12696657e-05 3.93562885e-05 3.24689820e-05 1.94894543e-05
 1.97957867e-05 2.37690598e-05 3.49393802e-05 2.29262272e-05
 2.39837870e-05 2.96319343e-05 4.42764622e-05 3.60607263e-05
 3.57427948e-05 1.72402498e-05 2.79007848e-05 6.01475456e-04
 1.96742727e-05 1.98476494e-05 2.81702

[6.12351785e-03 3.86624344e-05 2.34256956e-05 4.58279390e-05
 2.06129607e-05 3.16369345e-05 2.42625061e-01 3.13739024e-05
 3.86968450e-05 3.37929949e-04 1.63664235e-03 3.49706053e-05
 3.12342609e-05 2.36267912e-05 2.83873165e-05 3.11576225e-05
 3.16682435e-05 4.75192948e-05 2.72438205e-05 4.57894677e-05
 2.39493232e-05 2.22305381e-05 2.81929526e-05 2.80416138e-05
 1.98405612e-05 2.41596747e-05 2.79152493e-05 7.60086470e-04
 1.96240811e-05 2.78275803e-05 3.21932885e-05 4.68733521e-05
 3.14124827e-03 2.72004166e-05 2.16779452e-05]
[7.34793207e-03 4.25270337e-05 2.73289113e-05 3.05566272e-05
 2.06129607e-05 3.16369345e-05 2.72943099e-01 3.52942561e-05
 4.64326205e-05 2.53466928e-04 1.36392102e-03 3.49706053e-05
 2.34275250e-05 2.36267912e-05 2.83873165e-05 3.89439946e-05
 3.16682435e-05 4.35610786e-05 1.94613867e-05 3.05309718e-05
 2.39493232e-05 2.22305381e-05 3.22192196e-05 2.80416138e-05
 1.98405612e-05 2.41596747e-05 2.79152493e-05 8.86731312e-04
 2.35479731e-05 2.78275803e-05 3.21932

...

[8.27065017e-03 2.33491199e-05 3.97315735e-05 6.13522810e-05
 2.70985269e-05 3.58881725e-05 3.72128688e-01 2.69563577e-05
 2.71371473e-05 4.92535738e-04 4.68032600e-04 2.31161316e-05
 2.71972461e-05 2.77086162e-05 1.25440559e-04 2.74446305e-05
 2.62416057e-04 2.38550567e-05 3.09466982e-05 4.96336654e-05
 4.81096104e-04 4.45122019e-05 2.81136918e-05 2.38949578e-05
 1.97824719e-05 2.76072455e-05 2.38711911e-05 1.51521995e-03
 3.57427948e-05 3.93389467e-04 3.21325145e-05 4.34046617e-05
 2.56431565e-03 3.12574481e-05 7.60765533e-04]
[5.79013599e-03 2.33491199e-05 5.95855231e-05 5.75199690e-05
 2.37120142e-05 2.79152493e-05 3.72128688e-01 2.69563577e-05
 3.10126803e-05 4.10462494e-04 4.25500640e-04 3.08191341e-05
 2.33128310e-05 2.37511826e-05 1.17357118e-04 3.92020040e-05
 3.93577139e-04 1.98800043e-05 3.09466982e-05 3.43670168e-05
 4.37377485e-04 4.08043653e-05 4.41715623e-05 1.99132578e-05
 1.97824719e-05 4.14051529e-05 1.98934507e-05 1.85178342e-03
 3.97126393e-05 3.49693422e-04 3.21325

[6.30986623e-03 3.50182289e-05 6.35604162e-05 3.06702602e-05
 2.97942504e-04 1.19794433e-04 4.03094399e-01 5.78103743e-05
 3.87643477e-05 5.74535193e-04 2.97689236e-04 3.07681657e-05
 3.88622682e-05 4.35783360e-05 1.70048059e-04 5.08927767e-05
 3.93733669e-04 3.18459928e-05 1.93207594e-05 3.81608020e-05
 3.93061472e-04 2.22569265e-05 2.80866191e-05 3.18904245e-05
 2.77250781e-05 2.76034352e-05 5.17414995e-05 2.01904965e-03
 3.57740511e-05 3.49596177e-04 2.80967653e-05 2.36920975e-05
 2.56208509e-03 3.51976347e-05 5.43556402e-04]
[6.30986623e-03 3.50182289e-05 5.95902574e-05 2.68375066e-05
 2.97942504e-04 1.19794433e-04 4.03094399e-01 6.16620227e-05
 2.71381993e-05 4.10414362e-04 4.67725040e-04 2.69231805e-05
 4.27468338e-05 3.96182387e-05 1.66006017e-04 4.69797869e-05
 3.93733669e-04 2.38863963e-05 2.31840154e-05 3.43460325e-05
 5.24019548e-04 3.33816749e-05 4.41290333e-05 2.79052338e-05
 3.56437055e-05 2.07040052e-05 5.57193971e-05 1.68267948e-03
 2.78264741e-05 3.49596177e-04 2.80967

[1.26123088e-02 1.94955336e-05 2.77580607e-05 2.29665951e-05
 1.19015820e-03 5.19814946e-05 1.85767005e-01 6.16929311e-05
 4.27003715e-05 4.10783658e-04 8.07982402e-04 2.30265304e-05
 2.33482113e-05 1.98295452e-05 2.83574169e-05 3.13149541e-05
 3.94565360e-04 2.39388124e-05 3.47303957e-05 2.66272077e-05
 4.80122530e-04 4.09121171e-05 3.60203155e-05 4.78032419e-05
 2.37822497e-05 2.76062929e-05 2.78397544e-05 2.52547029e-03
 2.78010556e-05 3.06359140e-04 3.21557625e-05 7.90673219e-05
 2.24093843e-03 2.35000137e-05 5.52663868e-03]
[7.44738008e-03 2.33973772e-05 3.57045095e-05 2.68138620e-05
 2.23192395e-04 2.00016801e-05 5.56859712e-01 2.31250410e-05
 2.32929201e-05 7.39150329e-04 6.38102862e-04 3.83908108e-05
 1.94575999e-05 1.98232559e-05 2.83585657e-05 1.95672507e-05
 2.62823602e-04 3.18967820e-05 2.31553842e-05 3.04415161e-05
 5.23354704e-04 2.23215116e-05 3.20334429e-05 3.58567166e-05
 3.16983584e-05 2.41371819e-05 5.16489934e-05 1.51596695e-03
 2.78143117e-05 2.18732228e-04 2.00997

[5.79013599e-03 3.50988031e-05 1.98295452e-05 1.91439587e-05
 3.34562315e-04 1.99904845e-05 1.54514593e-01 1.92419444e-05
 3.49529494e-05 4.92959675e-04 2.12915039e-04 1.91946685e-05
 1.94492743e-05 1.98271862e-05 4.45255799e-05 1.95618919e-05
 2.63127469e-04 1.99553798e-05 2.31616412e-05 2.28051038e-05
 2.18220196e-04 1.85805224e-05 2.40472288e-05 2.39416781e-05
 2.37954543e-05 1.72420333e-05 1.98673655e-05 8.43347225e-04
 1.98728930e-05 2.18880209e-04 2.41148833e-05 2.37258244e-05
 2.15734386e-03 1.95756776e-05 2.05578184e-03]
[1.24093802e-02 3.89986701e-05 3.17260141e-05 2.68025685e-05
 2.97378675e-04 4.79829181e-05 2.78115925e-01 1.92441661e-05
 3.49515920e-05 4.92998251e-04 3.40571774e-04 3.07150070e-05
 1.94485178e-05 1.98264000e-05 5.66666262e-05 2.73812923e-05
 4.38598240e-04 2.39493232e-05 2.70198287e-05 2.66069657e-05
 3.92811938e-04 2.97288359e-05 3.60795193e-05 4.39035877e-05
 3.56861050e-05 2.41371819e-05 2.78098916e-05 1.68669445e-03
 3.57726292e-05 3.50194398e-04 3.21480

[5.79036299e-03 2.72926828e-05 2.77911219e-05 1.91073797e-05
 1.85949645e-04 6.00988024e-05 4.64274157e-01 2.31010014e-05
 3.49556646e-05 7.39208148e-04 3.40690440e-04 3.84085052e-05
 5.44346764e-05 1.98413486e-05 2.43024193e-05 2.73952231e-05
 3.07165738e-04 1.99370786e-05 3.08643166e-05 4.55946107e-05
 2.62093011e-04 2.97354659e-05 3.60448398e-05 3.59239848e-05
 2.37982857e-05 2.06605167e-05 3.57215151e-05 1.51802500e-03
 6.35402229e-05 2.18897632e-04 2.81249874e-05 3.16069361e-05
 2.88257368e-03 4.30496362e-05 6.49670449e-04]
[7.44650399e-03 2.72926828e-05 2.38237992e-05 2.67523762e-05
 2.23184848e-04 4.40883531e-05 4.64067014e-01 3.46421657e-05
 2.71887951e-05 4.92689813e-04 2.55596993e-04 3.84099804e-05
 3.50236799e-05 4.36423076e-05 2.42994666e-05 3.13100517e-05
 3.07190247e-04 3.58924662e-05 2.70094031e-05 6.83529595e-05
 3.93030263e-04 3.34436975e-05 2.80281403e-05 5.18860582e-05
 3.96732511e-05 3.44306378e-05 2.38048951e-05 8.43312532e-04
 1.98563197e-05 3.06481080e-04 4.01817